<a href="https://colab.research.google.com/github/sanskarmalviya7/Tree-Recognisition-and-Count/blob/main/Tree_recognition_for_Drone_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install DeepForest

In [2]:
from deepforest import main
from deepforest import get_data
from google.colab.patches import cv2_imshow
model = main.deepforest()
model.use_release()

Reading config file: /usr/local/lib/python3.10/dist-packages/deepforest/data/deepforest_config.yml


Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth
100%|██████████| 130M/130M [00:01<00:00, 79.2MB/s]


No validation file provided. Turning off validation loop


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


NEON.pt: 129MB [00:00, 144MB/s]                           


Model was downloaded and saved to /usr/local/lib/python3.10/dist-packages/deepforest/data/NEON.pt
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [3]:
raster_path = get_data("/content/drive/MyDrive/sanskar/Copy of phase1_Orthomosaic_export_MonMay01081306578090.jpg")

In [4]:
predicted_raster1 = model.predict_tile(raster_path, return_plot = False, patch_size=500,patch_overlap=0.25)

/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Predicting: |          | 0/? [00:00<?, ?it/s]

40024 predictions in overlapping windows, applying non-max supression
23927 predictions kept after non-max suppression


In [5]:
print(len(predicted_raster1))

23927


In [6]:
import pandas as pd
df = pd.DataFrame(predicted_raster1)
df.to_csv("object_detection_results.csv", index=False)

In [ ]:
import cv2
import pandas as pd
import numpy as np

# Function to check if a color is in the green range
def is_green(color):
    # Define the range of green color in HSV space
    lower_green = np.array([0, 50, 0], dtype=np.uint8)  # Lower bound of green color in HSV
    upper_green = np.array([0, 255, 0], dtype=np.uint8)  # Upper bound of green color in HSV

    # Convert color to HSV
    hsv = cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_BGR2HSV)[0][0]

    # Check if color falls within the green range
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Check if any non-zero elements exist in the mask
    return cv2.countNonZero(mask) > 0

# Specify file paths with either double backslashes or raw strings
raster_path = "/content/drive/MyDrive/sanskar/Copy of phase1_Orthomosaic_export_MonMay01081306578090.jpg"
df = pd.read_csv("object_detection_results.csv")

# Read the image
image = cv2.imread(raster_path)

# Check if the image data is loaded correctly
if image is None:
    print("Error: Unable to load image.")
    exit()

# Initialize counters for live and dead trees
live_tree_count = 0
dead_tree_count = 0

# Loop through each detected object and draw bounding boxes on the image
for index, row in df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])

    # Extract region within the bounding box
    region = image[ymin:ymax, xmin:xmax]

    # Calculate the average color of the region
    avg_color = np.mean(region, axis=(0, 1))

    # Check if the average color is green
    if is_green(avg_color):
        live_tree_count += 1
        color = (0, 255, 0)  # Green color for live trees
    else:
        dead_tree_count += 1
        color = (0, 0, 255)  # Red color for dead trees

    # Draw bounding box with the determined color
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color, thickness=2)

# Print total live and dead tree count
print("Total Trees:", len(df))
print("Total Live Trees:", live_tree_count)
print("Total Dead Trees:", dead_tree_count)

# Add text indicating the total number of trees detected, live and dead
total_tree_text = f'TOTAL TREES: {len(df)}, Live: {live_tree_count}, Dead: {dead_tree_count}'
cv2.putText(image, total_tree_text, (50, 500), cv2.FONT_HERSHEY_SIMPLEX, 10, (0, 0, 0), 10)

# Save the annotated image
output_image_path = "annotated_image_with_live_dead.jpg"
cv2.imwrite(output_image_path, image)

print("Annotated image with live and dead trees saved successfully.")

In [ ]:
# @title Work well for Good TIF
#from google.colab.patches import cv2_imshow
from deepforest import main
from deepforest import get_data
import matplotlib.pyplot as plt

model = main.deepforest()
model.use_release()

raster_path = get_data("/content/drive/MyDrive/sanskar/Copy of phase1_Orthomosaic_export_MonMay01081306578090.jpg")
predicted_raster = model.predict_tile(raster_path, return_plot = True, patch_size=300,patch_overlap=0.25, color=(255, 0, 0), thickness=2)
plt.figure(figsize=(40, 40))
plt.imshow(predicted_raster)
plt.show()
#image = cv2.imread(predicted_raster) #for real quality
#cv2_imshow(image)

In [ ]:
# @title no use
'''import cv2
import pandas as pd
from google.colab.patches import cv2_imshow
raster_path = "/content/drive/MyDrive/Stand by me/phase1_Orthomosaic_export_MonMay01081306578090.jpg"
# Read the object detection results from the CSV file
df = pd.read_csv("object_detection_results.csv")

# Read the image
image = cv2.imread(raster_path)

# Loop through each detected object and draw bounding boxes on the image
for index, row in df.iterrows():
    xmin = int(row['xmin'])
    ymin = int(row['ymin'])
    xmax = int(row['xmax'])
    ymax = int(row['ymax'])
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 0, 255), thickness=2)

# Add text indicating the total number of trees detected
cv2.putText(image, 'TOTAL TREES: ' + str(len(df)), (3, 14), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

# Display the annotated image

# Save the annotated image
output_image_path = "annotated_image.jpg"
cv2.imwrite(output_image_path, image)'''
